In [30]:
import torch
import numpy as np
from transformers import LlamaForCausalLM, LlamaTokenizer

In [9]:
device = 'cuda:3'

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("/data/LLAMA_hf/llama-13b")
model = LlamaForCausalLM.from_pretrained("/data/LLAMA_hf/llama-13b", torch_dtype=torch.float16)

In [14]:
model = model.to(device)

In [6]:
# for inference
for name, param in model.named_parameters():
    param.requires_grad = False

In [20]:
prompt_txt_file = './prompts/mimic/fewshot_gt-labels.txt' # path to prompt file
with open(prompt_txt_file, 'r') as f:
    prompt = f.read()

In [21]:
input_query = 'Edema' # depends on the input data

prompt = prompt.replace('<INPUT_QUERY>', input_query)
tokens = tokenizer(prompt, return_tensors='pt').to(device).input_ids
embs = model.model.embed_tokens(tokens)

In [23]:
outputs = model.generate( 
    inputs_embeds=embs,
    max_new_tokens=300,
    num_beams=1,
    do_sample=True,
    min_length=1,
    top_p=0.9,
    repetition_penalty=1,
    length_penalty=1,
    temperature=1,
) 

In [32]:
output_token = outputs[0]
if output_token[0] == 0:  # the model might output a unknow token <unk> at the beginning. remove it
    output_token = output_token[1:]
if output_token[0] == 1:  # some users find that there is a start token <s> at the beginning. remove it
    output_token = output_token[1:]
output_text = tokenizer.decode(output_token, add_special_tokens=False)

In [33]:
output_text

'"Mild perihilar, perilobular, and pericardial/pleural edema."\n\nConsolidation, Pneumonia\nReport:\n"1.  In the left basilar and lingual segments there is increased bronchial marking with small amount of mild peripheral consolidation.  2.  Perihilar and peribronchovascular interstitial marking concerning for pneumonia."\n\nFracture\nReport:\n"Inferior aspect of the right 1st and 2nd ribs are fractured.  2.  There is no evidence of lung contusion."\n\nLung Opacity\nReport:\n"There is minimal pleural effusion in the posterior lung fields, and scattered small opacities.  No evidence of pneumothorax or atelectasis."\n\nFracture\nReport:\n"1.  The right clavicle and 2nd rib are fractured.  2.  Inferior aspect of the right 1st and 2nd ribs are fractured."\n\nEdema\nReport:\n"No perihilar edema is noted."\n\nAtelectasis, Edema, Lung Opacity, Pleural Effusion\nReport:\n"There is minimal peribronchovascular pleural effusion and'